*Week 1 Velocity distributions:*

Background:
Irrespective of the start conditions, velocity distributions in multi-particle systems with
collisions between particles converge to a Maxwell-Boltzmann distribution. We will
investigate this effect by creating our own simulation of a multi-particle systems of
hard spheres in a 2D-box.

**Task I - Implementation**
In this first section will outline how we created an algorythm for simulating hard spheres in a box that includes 
* elastic collisions of hard spheres
* multi particle systems
* periodic boundary conditions for the box

All the code will be provided in easy to use functions that will be called for Task II.



Importing all external libraries used in this code

In [1]:
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

import copy
import numpy as np
import scipy as scy

Creating a particle class:

In [2]:
class Particle:
    """ A round particle with a defined position and velocity\n
    r   : the radius of the particle\n
    pos : the 2-dimensional position of the particle\n
    vel : the x- and y-component of the particle velocity
    """
    def __init__(self, r, x, y, vx, vy):
        """Initializing the Particle class.\n
        r       : radius of the particle\n
        x, y    : x and y positions. Don't need to be inside of a box-class\n
        vx, vy  : velocities in x- and y-direction
        """
        self.r = r
        self.pos = np.array([x,y])
        self.vel = np.array([vx,vy])
        
    def __repr__(self):
        """printing for debugging"""
        return str("This is a particle at %0.2f, %0.2f with v=%0.2f,%0.2f" % (self.pos[0],self.pos[1],self.vel[0],self.vel[1]))
    
    def move(self, dt = 1., vel = []):
        """moving the particle in the direction, where the velocity-vector points.\n
        dt  : the time-step moving forward; default = 1\n
        vel : a velocity vector for moving in that direction during a time-step of one; default = self.vel
        """
        if len(vel) == 0:
            vel = self.vel
        self.pos += vel*dt

    def calculate_distance(self, particle_1):
        """Calculate the distance to another particle.\n
        particle_1  : The other particle
        """
        return np.linalg.norm(self.pos)



In [11]:
class Box:
    """Box-class: defining a rectangular box-shape, in which Particles can roam. One corner is always (0,0)\n
    box         : 2-dimensional array of the two lengths of the x- and y-axis, corresponding to the boundaries of the box.\n
    particles   : a list of all particles in the box\n
    n_particles : the number of particles inside of the box
    """
    def __init__(self,box_size):
        """Initializing the Box-class\n
        box_size    : a 2-dimensional array of the two lengths of the x- and y-axis, corresponding to the boundaries of the box.
        """
        self.box = box_size
        self.particles = []
        self.n_particles = 0
        
    def __repr__(self):
        """printing for debugging"""
        return str("This is a box of size %0.2f" % (self.box) + ", with %0.2f" % (self.n_particles) + " particles")



    def random_positions(self, axis, n_particles = 0):
        """return random positions for a number of particles (only one axis)\n
        axis        : either 0 or 1.     0 = x-axis, 1 = y-axis\n
        n_particles : the number of particles for which positions should be given; default self.n_particles\n
        returns     : array of random positions
        """
        if n_particles == 0:
            n_particles = self.n_particles
        rnd = np.random.randint(1,self.box[axis]-1,n_particles)
        return rnd



    def fill_particles(self, n_particles, radius, vel, angle = 0, x = [], y = []):
        """fills the particles-array with particles\n
        n_particles : The amount of particles that should be inserted\n
        radius      : The radius of particles; either as array of length n for individual radii or int/float for a general radius
        vel         : The absolute velocity; either as array of length n for individual velocities or int/float for a uniform initial velocity
        angle       : The initial angles of the particles as array of length n for individual angles; default is uniformly distributed
        x,y         : initial positions as array of length n; default random positions 0.5 away from border
        """
        if type(radius) == int or type(radius) == float:
            radius = np.ones(n_particles)*radius
        if type(vel) == int or type(vel) == float:
            vel = np.ones(n_particles)*vel
        if angle == 0:
            angle = np.random.uniform(0,2 * np.pi, n_particles)
        else:
            angle = np.array(angle)
        
        if len(x) == 0:
            x = self.random_positions(0,n_particles)
        if len(y) == 0:
            y = self.random_positions(1,n_particles)

        for i in range(n_particles):
            self.particles.append(Particle(radius[i],x[i],y[i],np.sin(angle[i]) * vel[i],np.cos(angle[i]) * vel[i]))



    def wrap_around(self, particles = []):
        """For continuous borders, i.e. a particle that exits to the right is entering from the left and vice versa\n
        particles   : Particles, which should be wrapped; default self.particles\n
        returns     : array of particles with new positions
        """
        if len(particles) == 0:
            particles = self.particles
        for i in range(len(particles)):
            particles[i].pos = particles[i].pos % self.box
        return particles



    def reflect(self, particles = []):
        """Reflecting particles on the edges of the box.\n
        particles   : Particles, which should be reflected; default self.particles\n
        returns     : array of particles with new positions
        """
        if len(particles) == 0:
            particles = self.particles
        
        for i in range(len(particles)):
            if (particles[i].pos[0] + particles[i].r) >= self.box[0] or (particles[i].pos[0]) <= particles[i].r:
                particles[i].vel[0] *= (-1)
            if (particles[i].pos[1] + particles[i].r) >= self.box[1] or (particles[i].pos[1]) <= particles[i].r:
                particles[i].vel[1] *= (-1)

        return particles




In [20]:
class Simulation:
    """Simulation class for everything related to simulating the particles in a box\n
    box         : The box object which should be simulated
    steps       : The number of integration steps to perform
    dt          : The length of one time step
    data_traj   : The trajectories of all particles
    """
    def __init__(self, box, steps, dt):
        """Initializing the Simulation\n
    box         : The box object which should be simulated
    steps       : The number of integration steps to perform
    dt          : The length of one time step
    """
        self.box = box
        self.steps = steps
        self.dt = dt
        self.data_traj = np.array([])

    def set_traj(self):
        """Initialize the trajectory-saving"""
        self.data_traj = np.zeros((self.box.n_particles,2,self.steps))



    def collision(self, particle_1, particle_2):
        """Collide two particles without considering wether overlap exists.\n
        particles   : two particles that collide.
        """
        # calculate angle between x-axis and line between particles
        phi = np.arctan((particle_2.pos[1] - particle_1.pos[1])/(particle_2.pos[0] - particle_1.pos[0]))
        # change the velocities
        sin_phi = np.sin(phi)
        cos_phi = np.cos(phi)
        # Get the velocities of particles i and j as variables
        v1x, v1y = particle_1.vel[0], particle_1.vel[1]
        v2x, v2y = particle_2.vel[0], particle_2.vel[1]

        # Calculate the updated velocities using the provided formulas: https://hermann-baum.de/elastischer_stoss/
        # the tangental part stays the same, the normal part changes. This is done in transformed coordinates and then transformed directly back
        new_v1x = ( v1x * sin_phi - v1y * cos_phi) * sin_phi + (v2x * cos_phi + v2y * sin_phi) * cos_phi
        new_v1y = (-v1x * sin_phi + v1y * cos_phi) * cos_phi + (v2x * cos_phi + v2y * sin_phi) * sin_phi
        new_v2x = ( v2x * sin_phi - v2y * cos_phi) * sin_phi + (v1x * cos_phi + v1y * sin_phi) * cos_phi
        new_v2y = (-v2x * sin_phi + v2y * cos_phi) * cos_phi + (v1x * cos_phi + v1y * sin_phi) * sin_phi

        # Update the particles' velocities
        particle_1.vel[0], particle_1.vel[1] = new_v1x, new_v1y
        particle_2.vel[0], particle_2.vel[1] = new_v2x, new_v2y
        return particle_1, particle_2

    def collide_simple(self, particles):
        """Initial idea on how collision works.\n
        particles   : All particles for which the collision should happen
        """
        for i in range(len(particles)): # looping through all particles
            for j in range(i,len(particles)):
                if particles[i].calculate_distance(particles[j])<(particles[i].r+particles[j].r):
                    particles[i], particles[j] = self.collision(particles[i], particles[j])


    def check_overlap(self, particle_1, particle_2):
        """Check if two particles overlap\n
        particles   : two particles to check wether they overlap
        """
        # how far apart the particles should be on impact
        R = particle_1.r+particle_2.r
        if particle_1.calculate_distance(particle_2)<R:
            return True
        else:
            return False

    def collision_complex(self, particle_1, particle_2):
        """The more complex version of collisions. This includes time-warping two colliding particles backwards, so that they don't overlap. After the collision, time is warped forward again.\n
        particles   : two particles that get checked if they should collide
        """
        if self.check_overlap(particle_1,particle_2):
            #print(step)
            print(particle_1, particle_2,'collided with a distance of ', particle_1.calculate_distance(particle_2))
            
            # in time
            # defining re-used variables
            velocity_vector = particle_1.vel-particle_2.vel     # von j nach i
            distance_vector = particle_2.pos-particle_1.pos     # von i nach j
            R = particle_1.r+particle_2.r

            #calculating the time needed to travel back in two steps:
            sqrt = 2*np.sqrt((distance_vector[0]*velocity_vector[0]+distance_vector[1]*velocity_vector[1])**2-(velocity_vector[0]**2+velocity_vector[1]**2)*(distance_vector[0]**2+distance_vector[1]**2-R**2))
            
            delta_t = -1*(-2*(distance_vector[0]*velocity_vector[0]+distance_vector[1]*velocity_vector[1]) + sqrt)/(2*(velocity_vector[0]**2+velocity_vector[1]**2))
            
            #solving the quadratic equation results in two solutions (one positive and one negative), we want the negative solution, for backwards time-travel
            if delta_t > 0:
                delta_t = -1*(-2*(distance_vector[0]*velocity_vector[0]+distance_vector[1]*velocity_vector[1]) - sqrt)/(2*(velocity_vector[0]**2+velocity_vector[1]**2))

            # rewind time to just outside of the collision
            particle_1.move(delta_t)
            particle_2.move(delta_t)

            # change the velocities
            particle_1, particle_2 = self.collision(particle_1, particle_2)
            
            # finish this time_step, that was rewound previously and open the boundaries again with np.mod
            particle_1.move(-delta_t)
            particle_2.move(-delta_t)
            



                

    def collide_border_check(self):
        """Check if there are particles that might collide with particles on the opposite side of the box"""
        #run through all pairwise possible interactions and check collisions
        for i in range(len(self.box.particles)):
            # account for edge cases
            if self.box.particles[i].pos[0] < self.box.particles[i].r:
                self.box.particles[i].pos[0] += self.box[0]
            elif self.box.particles[i].pos[0] < self.box.box[0]-self.box.particles[i].r:
                self.box.particles[i].pos[0] -= self.box[0]

            if self.box.particles[i].pos[1] < self.box.particles[i].r:
                self.box.particles[i].pos[1] += self.box[1]
            elif self.box.particles[i].pos[1] < self.box.box[1]-self.box.particles[i].r:
                self.box.particles[i].pos[1] -= self.box[1]

            for j in range(i,len(self.box.particles)):
                self.collision_complex(self.box.particles[i],self.box.particles[j])



    def check_collisions(self):
        for i in range(self.box.n_particles):
            for j in range(self.box.n_particles):
                self.collision_complex(self.box.particles[i], self.box.particles[j])
                self.collide_border_check()
                self.box.wrap_around([self.box.particles[i], self.box.particles[j]])

# self.box.wrap_around([particle_1,particle_2])                

    def run(self):
        for i in range(self.steps):
            for j in range(self.box.n_particles):
                #move(p1,1)
                #reflect(p1)
                self.box.particles[j].move_unbound(self.dt)
            self.check_collisions()
            for j in range(self.box.n_particles):
                self.data_traj[j,:,i] = [self.box.particles[j].pos[0], self.box.particles[j].pos[1], self.box.particles[j].vel[0], self.box.particles[j].vel[1]]
                


In [21]:
# defining the box
box_size = [20,20]

box = Box(box_size)

In [22]:
number_of_particles = 50
radius = 0.5
vel = np.ones(number_of_particles)*0.5
angles = np.random.uniform(0,2*np.pi,number_of_particles)

box.fill_particles(number_of_particles,radius,vel)



In [23]:
# Defining number of simulation steps
steps = 50
sim = Simulation(box, steps, 1)
sim.set_traj()

In [24]:
sim.run()

In [26]:
sim.steps

50

Define box reflection:

Define a function that calculates new velocities for two particles (with the same mass) colliding with two different velocities in 2D. The formulas can be found here: 
 https://hermann-baum.de/elastischer_stoss/, 24.10.2024.


The following algorithm works by checkin pariwise collisions. If there is a collision, defined by any overlap of two particles, the particles travel back in time, so that they are just touching. Then the new velocities are calculated whilst turning the coordinate system, to provide for easier calculation. Then the time is unwound forwards, considering the new velocities, so that each particle has the same time passed in one calculation step. When a particle is in the boundary of the box (close to 0 position on either axis), it is additionally checked for any particles that are on the opposite side of the box.

**Task II - Carrying out the simulation**
In the following the above functions will be implemented to run a simulation.

The actual integration happens here. Everything is saved to the predefined data_traj-array


In [ ]:

for i in range(steps):
    for j in range(number_of_particles):
        #move(p1,1)
        #reflect(p1)
        move_unbound(particles[j],1)
        collide_boundary(particles,data_traj,i,dt)    
        for j in range(number_of_particles):
            data_traj[j,:,i] = [particles[j].x, particles[j].y, particles[j].vx, particles[j].vy]
        

The following code is just for visualisation purposes.

Plotting trajectory with Matplotlib:

In [ ]:
for data_traj_j in data_traj:
    plt.plot(data_traj_j[0],data_traj_j[1],'-')
plt.xlabel('position x')
plt.ylabel('position y')

plt.grid()
plt.show()

**Animate trajectory:**

* Set up the figure, the axis, and the plot element we want to animate

In [ ]:
fig, ax = plt.subplots()

ax.set_xlim((0, box[0]))
ax.set_ylim((0, box[1]))

plt.xlabel('position x')
plt.ylabel('position y')

# make the points in the plot the correct size according to the radius of the particles
# use ax.transData.transform to convert radius size from particle to markerSize from pyplot
desired_size_in_data_units = radius
data_to_points = ax.transData.transform((desired_size_in_data_units, 0)) - ax.transData.transform((0, 0))
marker_size_in_points = data_to_points[0]


dot, = ax.plot([], [], 'bo', ms=marker_size_in_points)


#ax.plot(5,5, 'bo', ms=marker_size_in_points)
# dont show the plot 
#plt.close()

In [18]:
# * initialization function: plot the background of each frame
def init():
    dot.set_data([], [])
    return (dot,)

In [19]:
# animation function. This is called sequentially
def animate(i):
    x = data_traj[:,0,i]
    y = data_traj[:,1,i]
    dot.set_data(x, y)
    return (dot,)


* call the animator. blit=True means only re-draw the parts that have changed.

In [20]:
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=steps, interval=60, blit=True)

In [21]:
HTML(anim.to_html5_video())

# uncomment if video should not be saved
anim.save('animation.mp4', writer='ffmpeg', fps=30)


***Task III - analysis of the velocity distributions:***

Finally the results from the above simulation can be analysed.

Analysis of the velocity distribution in x direction
In order to plot a histogram the data saved in the data_traj has to be transformed into a list.

In [ ]:
# format of data_traj:

list_for_histogramm = []
for 0 in range(steps):
    for 1 in range(number_of_particles):
        list_for_histogramm.append((data_traj)[1, 0, 0])

plt.hist(list_for_histogramm,range=(0,box[0]),bins=10)
plt.xlabel('position x')
plt.ylabel('occupancy')
plt.show()

In [ ]:
# histogram of the y positions

list_for_histogramm = []
for 0 in range(steps):
    for 1 in range(number_of_particles):
        list_for_histogramm.append((data_traj)[1, 1, 0])

plt.hist(list_for_histogramm,range=(0,box[0]),bins=10)
plt.xlabel('position y')
plt.ylabel('occupancy')
plt.show()

In [ ]:
# 2d plot of the position distribution

a = []
for 0 in range(steps):
    for 1 in range(number_of_particles):
        a.append((data_traj)[1, 0, 0])
b = []
for 0 in range(steps):
    for 1 in range(number_of_particles):
        b.append((data_traj)[1, 1, 0])
    

plt.hist2d(a, b, range=[[0,box[0]],[0,box[1]]], bins=10)
plt.xlabel('position x')
plt.ylabel('position y')
plt.colorbar().set_label('occupancy')
plt.show()

In the above plots it can be observed that whilst the simple histograms of x and y distribution show an evenly spread distribution, in 2D the box is not homogeniously sampled. Certain spots show a significantly higher occupancy than others. This is clearly a limitation.

In [ ]:
# comparison of different iteration parts

# first 10%
a = []
for 0 in range(np.int(0.1 *steps)):
    for 1 in range(number_of_particles):
        a.append(data_traj[1,2,0])

# last 10%
b = []
i_0 = np.int(0.9 * steps)-1
for 0 in range(np.int(0.1 * steps)):
    0 = 0+i_0
    
    for 1 in range(number_of_particles):
        b.append(data_traj[1,2,0])
    0 = 0+1

plt.hist(a,bins=10, label = 'first 10% iterations')
plt.hist(b,bins=10, label = 'last 10% iterations', alpha=0.5)
plt.xlabel('velocity x')
plt.ylabel('occupancy')
plt.legend()
plt.show()


Analysis of the velocity distribution in the x direction

In [ ]:
# plt.hist(data_traj[1],range=(0,box[1]),bins=3)
# plt.xlabel('position y')
# plt.ylabel('occupancy')
# plt.show()

# histogram of the x velocity

a = []
for 0 in range(steps):
    for 1 in range(number_of_particles):
        a.append((data_traj)[1, 2, 0])
    
plt.hist(a,bins=50)
plt.xlabel('velocity x')
plt.ylabel('occupancy')
plt.show()

In [ ]:
# histogram of the y velocity

a = []
for 0 in range(steps):
    for 1 in range(number_of_particles):
        a.append((data_traj)[1, 3, 0])
    
plt.hist(a,bins=50)
plt.xlabel('velocity y')
plt.ylabel('occupancy')
plt.show()

In [26]:
# Maxwell Boltzmann Distribution
def mwb_dist(vs, const):
    # const = kT / m
    return 4 * np.pi * (2 * np.pi * const)**(-3/2) * vs**2 * np.exp(-0.5 * vs**2 / const)

In [ ]:
# Histogram of |velocity|

######################################
NOT WORKING!
######################################

0 = 0
all_velocities = []
for 0 in range(steps):
    for 1 in range(number_of_particles):
        print(data_traj[0, 2, 1])
        print(data_traj[0, 3, 1])
        
        velocity = np.sqrt((data_traj[0, 2, 1])**2 + (data_traj[0, 3, 1])**2) 
        # all_velocities.append(velocity)


plt.hist(velocity,bins=100)
plt.xlabel('velocity |v|')
plt.ylabel('occupancy')
plt.show()



Analysis of the velocity distribution in the xy Plane

**Interpretation:**
* in the reduced data projection, the occupancy of each bin seems even and well-distributed
* in the full dimensional projection, it can be seen easily that the box is **not** fully sampled, large undersampled patches present
* *data dimensionality reduction always brings the danger of wrong projection!*

**Solution for improvements:**
* enhance sampling by
    * prolong the simulation
    * increase the time step
* *Caveat: all solutions come with problems like increased computational cost or sampling errors!*